#14MBID - TFM

#Análisis de la influencia de los factores deportivos en la tipología y duración de las lesiones en La Liga.

### Jokin Atxa Galdos

#### Cuaderno1

En este cuaderno se van a generar los datasets de los jugadores que hayan jugado en la liga, las lesiones de cada uno y los partidos que haya jugado cada uno de ellos.

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from google.colab import files


In [ ]:
headers = {'User-Agent':
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
base = "https://www.transfermarkt.es"

!mkdir PlayerInjuries

## Funcion lesiones de los jugadores

In [ ]:
def GetInjuryPage(page, Injury, Values):
  pageTree = requests.get(base + page, headers=headers)
  pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
  Injury.extend( pageSoup.find_all("td", {"class": "hauptlink"}))
  Values.extend( pageSoup.find_all("td", {"class": "zentriert"}))

  hasNextLink = pageSoup.find_all("li", {"class": "tm-pagination__list-item tm-pagination__list-item--icon-next-page"})
  if (len(hasNextLink) > 0):
    children = hasNextLink[0].findChildren("a" , recursive=False)[0]
    link = children['href']
    GetInjuryPage(link, Injury, Values)


def GetInjuriesFromPlayer(url, name):
  if "profil" in url:
    url = url.replace('profil','verletzungen')
  Injury = []
  Values = []

  GetInjuryPage(url, Injury, Values)

  InjuryList = []
  FromList = []
  ToList = []
  TeamsList = []
  NumberOfMatchesList = []


  for i in range(0,int(len(Injury)),2): #cada 2 porque el numero de dias tiene el mismo html class
    InjuryList.append(Injury[i].text)
    f = (int(i/2) * 3) + 1
    t = (int(i/2) * 3) + 2
    FromList.append(Values[f].text)
    ToList.append(Values[t].text)
    teams = Injury[i+1].find_all("a")
    teamsHePlayed = []
    for t in teams:
      teamsHePlayed.append(t['title'])
    TeamsList.append(teamsHePlayed)
    NumberOfMatchesList.append(Injury[i+1].text)

  df = pd.DataFrame({"Injury":InjuryList,"From":FromList , "To": ToList, "Teams" : TeamsList , "NumberOfMatches" : NumberOfMatchesList })
  df['Player'] = name
  df.to_json(f"PlayerInjuries/{name}-{url.split('/')[-1]}.json")


## Conseguir equipos de La Liga

In [ ]:
fromSeason = 2010
toSeason = 2022

Teams = {}

pageSeason  = "/laliga/startseite/wettbewerb/ES1/plus/?saison_id="
for i in range(fromSeason,toSeason+1):
  innerPageSeason = f"{pageSeason}{i}"
  pageTree = requests.get(base + innerPageSeason, headers=headers)
  pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
  mainContainer = pageSoup.find("div", {"class": "responsive-table"})
  innerTeams = mainContainer.find_all("td", {"class": "hauptlink"})
  for t in innerTeams:
    name = t.text.strip()
    if name in Teams:
      Teams[name].append(t.find("a")['href'])
    else:
      Teams[name] = [t.find("a")['href']]

for t in Teams:
  print(f"{t} = {Teams[t]}")


FC Barcelona = ['/fc-barcelona/startseite/verein/131/saison_id/2010', '/fc-barcelona/startseite/verein/131/saison_id/2011', '/fc-barcelona/startseite/verein/131/saison_id/2012', '/fc-barcelona/startseite/verein/131/saison_id/2013', '/fc-barcelona/startseite/verein/131/saison_id/2014', '/fc-barcelona/startseite/verein/131/saison_id/2015', '/fc-barcelona/startseite/verein/131/saison_id/2016', '/fc-barcelona/startseite/verein/131/saison_id/2017', '/fc-barcelona/startseite/verein/131/saison_id/2018', '/fc-barcelona/startseite/verein/131/saison_id/2019', '/fc-barcelona/startseite/verein/131/saison_id/2020', '/fc-barcelona/startseite/verein/131/saison_id/2021', '/fc-barcelona/startseite/verein/131/saison_id/2022']
Real Madrid CF = ['/real-madrid/startseite/verein/418/saison_id/2010', '/real-madrid/startseite/verein/418/saison_id/2011', '/real-madrid/startseite/verein/418/saison_id/2012', '/real-madrid/startseite/verein/418/saison_id/2013', '/real-madrid/startseite/verein/418/saison_id/2014',

In [ ]:
toAddTeams = []
TeamsSeasons = []
for t in Teams:
  toAddTeams.append(t)
  TeamsSeasons.append(Teams[t])
print(Teams)
teamsDf = pd.DataFrame({"Team":toAddTeams,"Seasons":TeamsSeasons})
teamsDf.to_json(f"FirstDivisionTeams.json")

{'FC Barcelona': ['/fc-barcelona/startseite/verein/131/saison_id/2010', '/fc-barcelona/startseite/verein/131/saison_id/2011', '/fc-barcelona/startseite/verein/131/saison_id/2012', '/fc-barcelona/startseite/verein/131/saison_id/2013', '/fc-barcelona/startseite/verein/131/saison_id/2014', '/fc-barcelona/startseite/verein/131/saison_id/2015', '/fc-barcelona/startseite/verein/131/saison_id/2016', '/fc-barcelona/startseite/verein/131/saison_id/2017', '/fc-barcelona/startseite/verein/131/saison_id/2018', '/fc-barcelona/startseite/verein/131/saison_id/2019', '/fc-barcelona/startseite/verein/131/saison_id/2020', '/fc-barcelona/startseite/verein/131/saison_id/2021', '/fc-barcelona/startseite/verein/131/saison_id/2022'], 'Real Madrid CF': ['/real-madrid/startseite/verein/418/saison_id/2010', '/real-madrid/startseite/verein/418/saison_id/2011', '/real-madrid/startseite/verein/418/saison_id/2012', '/real-madrid/startseite/verein/418/saison_id/2013', '/real-madrid/startseite/verein/418/saison_id/20

## Conseguir los jugadores que hayan jugado en esos equipos

In [ ]:
i = 0
PlayerList = {}
PlayerYearList = []

for t in Teams:
  for s in Teams[t]:
    i = i + 1
    print(f"{i} = {s}")
    pageTree = requests.get(base + str(s), headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    mainContainer = pageSoup.find("div", {"class": "responsive-table"})
    innerPlayers = mainContainer.find_all("td", {"class": "hauptlink"})
    for p in innerPlayers:
      td = p.find_all("a")
      for t in td:
        if ("profil" in t["href"]):
          PlayerYearList.append(t.text.strip()+"_"+s[-4:])
          if t["href"] not in Teams:
            PlayerList[t["href"]] = t.text.strip()

print(len(PlayerList))
# for p in PlayerList:
#   print(p)

1 = /fc-barcelona/startseite/verein/131/saison_id/2010
2 = /fc-barcelona/startseite/verein/131/saison_id/2011
3 = /fc-barcelona/startseite/verein/131/saison_id/2012
4 = /fc-barcelona/startseite/verein/131/saison_id/2013
5 = /fc-barcelona/startseite/verein/131/saison_id/2014
6 = /fc-barcelona/startseite/verein/131/saison_id/2015
7 = /fc-barcelona/startseite/verein/131/saison_id/2016
8 = /fc-barcelona/startseite/verein/131/saison_id/2017
9 = /fc-barcelona/startseite/verein/131/saison_id/2018
10 = /fc-barcelona/startseite/verein/131/saison_id/2019
11 = /fc-barcelona/startseite/verein/131/saison_id/2020
12 = /fc-barcelona/startseite/verein/131/saison_id/2021
13 = /fc-barcelona/startseite/verein/131/saison_id/2022
14 = /real-madrid/startseite/verein/418/saison_id/2010
15 = /real-madrid/startseite/verein/418/saison_id/2011
16 = /real-madrid/startseite/verein/418/saison_id/2012
17 = /real-madrid/startseite/verein/418/saison_id/2013
18 = /real-madrid/startseite/verein/418/saison_id/2014
19 = /

In [ ]:
# Descargar elementos cuando la lista esté completa
if len(PlayerYearList) > 0:

    for jugadores in PlayerYearList:
        print(jugadores)
        with open("JugadoresAño.txt", "a") as file:
            file.write(jugadores + "\n")

    # Limpiar la lista después de descargar los elementos
    PlayerYearList = []


Se han truncado las últimas 5000 líneas del flujo de salida.
Duda_2016
Pablo Fornals_2016
Juanpi_2016
Adalberto Peñaranda_2016
Javi Ontiveros_2016
Jony_2016
Chory Castro_2016
Álex Mula_2016
Keko Gontán_2016
Kuki Zalazar_2016
Sandro Ramírez_2016
Michael Santos_2016
Charles_2016
Youssef En-Nesyri_2016
Roberto_2017
Andrés Prieto_2017
Cenk Gönen_2017
Luis Hernández_2017
Paul Baysse_2017
Diego González_2017
Ignasi Miquel_2017
Federico Ricca_2017
Juankar_2017
Alberto López_2017
Roberto Rosales_2017
Miguel Torres_2017
Iván Rodríguez_2017
Álex Robles_2017
Emanuel Cecchini_2017
Manuel Iturra_2017
Esteban Rolón_2017
Mehdi Lacen_2017
Ian Soler_2017
Adrián González_2017
Recio_2017
Zdravko Kuzmanovic_2017
José Carlos Sánchez_2017
Juanpi_2017
Javi Ontiveros_2017
Maxime Lestienne_2017
Adalberto Peñaranda_2017
Jony_2017
Chory Castro_2017
Álex Mula_2017
Keko Gontán_2017
Samu García_2017
Miguel Cifuentes_2017
Juan Cruz_2017
Alberto Bueno_2017
Diego Rolán_2017
Borja Bastón_2017
Brown Ideye_2017
Isaac Suc

In [ ]:
i = 0
for p in PlayerList:
  i = i+1
  print(f"{i}/{len(PlayerList)} : {PlayerList[p]} => {p}")
  GetInjuriesFromPlayer(p,PlayerList[p])

1/3153 : Víctor Valdés => /victor-valdes/profil/spieler/7589
2/3153 : Rubén Miño => /ruben-mino/profil/spieler/65266
3/3153 : Oier Olazábal => /oier-olazabal/profil/spieler/8176
4/3153 : José Manuel Pinto => /jose-manuel-pinto/profil/spieler/7537
5/3153 : Gerard Piqué => /gerard-pique/profil/spieler/18944
6/3153 : Carles Puyol => /carles-puyol/profil/spieler/7594
7/3153 : Gabriel Milito => /gabriel-milito/profil/spieler/7968
8/3153 : Marc Muniesa => /marc-muniesa/profil/spieler/74228
9/3153 : Andreu Fontàs => /andreu-fontas/profil/spieler/65237
10/3153 : Marc Bartra => /marc-bartra/profil/spieler/99922
11/3153 : Sergi Gómez => /sergi-gomez/profil/spieler/92947
12/3153 : Éric Abidal => /eric-abidal/profil/spieler/5283
13/3153 : Adriano => /adriano/profil/spieler/34495
14/3153 : Maxwell => /maxwell/profil/spieler/4317
15/3153 : Dani Alves => /dani-alves/profil/spieler/15951
16/3153 : Martín Montoya => /martin-montoya/profil/spieler/68645
17/3153 : Sergio Busquets => /sergio-busquets/prof

Descargar los datos

In [ ]:
!zip -r PlayerInjuries.zip PlayerInjuries/

updating: PlayerInjuries/ (stored 0%)
updating: PlayerInjuries/Cristian Álvarez-55135.json (deflated 47%)
updating: PlayerInjuries/Ricard Sánchez-388545.json (deflated 28%)
updating: PlayerInjuries/Javi Acuña-35532.json (deflated 56%)
updating: PlayerInjuries/Javi Varas-58341.json (deflated 27%)
updating: PlayerInjuries/Iñigo Muñoz-442767.json (deflated 27%)
updating: PlayerInjuries/Florian Lejeune-127108.json (deflated 62%)
updating: PlayerInjuries/Chengdong Zhang-138655.json (deflated 44%)
updating: PlayerInjuries/Welliton-45593.json (deflated 70%)
updating: PlayerInjuries/Diego Piñeiro-926850.json (deflated 27%)
updating: PlayerInjuries/Aarón Martín-251878.json (deflated 45%)
updating: PlayerInjuries/Chuli-129515.json (deflated 44%)
updating: PlayerInjuries/Gonzalo Pereira-536152.json (deflated 27%)
updating: PlayerInjuries/Alberto Ródenas-461623.json (deflated 27%)
updating: PlayerInjuries/Asier del Horno-7617.json (deflated 54%)
updating: PlayerInjuries/Damián Musto-67823.json (de

In [ ]:
files.download('PlayerInjuries.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Extraer la informacion de cada uno de los jugadores

In [ ]:
def obtener_informacion_jugador(url,izena):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        info_table = soup.find('div', class_='large-6 large-pull-6 small-12 columns spielerdatenundfakten')

        jugador_info = {
            "Izena" : izena,
            "Nombre": info_table.find('span', class_='info-table__content--bold').text.strip() if info_table.find('span', class_='info-table__content--bold') else "",
            "Fecha de Nacimiento": info_table.find('span', text='F. Nacim./Edad:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='F. Nacim./Edad:') else "",
            "Lugar de Nacimiento": info_table.find('span', text='Lugar de nacimiento:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Lugar de nacimiento:') else "",
            "Altura": info_table.find('span', text='Altura:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Altura:') else "",
            "Nacionalidad": info_table.find('span', text='Nacionalidad:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Nacionalidad:') else "",
            "Posición": info_table.find('span', text='Posición:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Posición:') else "",
            "Pie": info_table.find('span', text='Pie:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Pie:') else "",
            "Agente": info_table.find('span', text='Agente:').find_next('a').text.strip() if info_table.find('span', text='Agente:') else "",
            "Proveedor": info_table.find('span', text='Proveedor:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Proveedor:') else "",
        }

        return jugador_info

    else:
        print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")
        return None

In [ ]:
# Inicializar una lista para almacenar la información de todos los jugadores
lista_jugadores = []

# Iterar a través de las URLs y obtener la información de cada jugador
for p in PlayerList:
    print(base+p)
    info_jugador = obtener_informacion_jugador(base+p,PlayerList[p])
    if info_jugador:
        lista_jugadores.append(info_jugador)

# Convertir la lista en un DataFrame de pandas
df = pd.DataFrame(lista_jugadores)

# Guardar el DataFrame como un archivo Excel
df.to_excel("informacion_jugadores.xlsx", index=False)

print("Información guardada correctamente en informacion_jugadores.xlsx")

https://www.transfermarkt.es/victor-valdes/profil/spieler/7589


<ipython-input-9-3fc2dfb4b3e1>:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  "Fecha de Nacimiento": info_table.find('span', text='F. Nacim./Edad:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='F. Nacim./Edad:') else "",
<ipython-input-9-3fc2dfb4b3e1>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  "Lugar de Nacimiento": info_table.find('span', text='Lugar de nacimiento:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Lugar de nacimiento:') else "",
<ipython-input-9-3fc2dfb4b3e1>:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  "Altura": info_table.find('span', text='Altura:').find_next('span', class_='info-table__content--bold').text.strip() if info_table.find('span', text='Altura:') else "",
<ipython-input

https://www.transfermarkt.es/ruben-mino/profil/spieler/65266
https://www.transfermarkt.es/oier-olazabal/profil/spieler/8176
https://www.transfermarkt.es/jose-manuel-pinto/profil/spieler/7537
https://www.transfermarkt.es/gerard-pique/profil/spieler/18944
https://www.transfermarkt.es/carles-puyol/profil/spieler/7594
https://www.transfermarkt.es/gabriel-milito/profil/spieler/7968
https://www.transfermarkt.es/marc-muniesa/profil/spieler/74228
https://www.transfermarkt.es/andreu-fontas/profil/spieler/65237
https://www.transfermarkt.es/marc-bartra/profil/spieler/99922
https://www.transfermarkt.es/sergi-gomez/profil/spieler/92947
https://www.transfermarkt.es/eric-abidal/profil/spieler/5283
https://www.transfermarkt.es/adriano/profil/spieler/34495
https://www.transfermarkt.es/maxwell/profil/spieler/4317
https://www.transfermarkt.es/dani-alves/profil/spieler/15951
https://www.transfermarkt.es/martin-montoya/profil/spieler/68645
https://www.transfermarkt.es/sergio-busquets/profil/spieler/65230
h

##Extraer los partidos jugados por cada Jugador

In [ ]:
def GetGamesFromPlayer(urloriginal, izena):
  urloriginal = urloriginal.replace('profil','leistungsdaten')
  urloriginal = urloriginal+"/plus/1?saison="
  url2 = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]
  for urls in url2:
    url = urloriginal+urls
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        html_content = response.content
    else:
        print(f"Error al obtener la página: {response.status_code}")
        continue

    soup = BeautifulSoup(html_content, "lxml")

    # Encontrar todos los "box" en la página
    boxes = soup.find_all("div", class_="box")

    # Crear un directorio para almacenar los archivos
    output_directory = "datos_tablas"
    os.makedirs(output_directory, exist_ok=True)

    # Definir los títulos
    column_titles = ["Jornada", "Fecha","Vacio1", "Local","Vacio2","Visitante", "Resultado", "Pos", "Gol", "Asistencia","Propia", "Tarjeta Amarilla", "Doble amarilla","Roja","Entra","Sale", "Minutos"]

    # Iterar sobre cada box y guardar la tabla en un archivo separado
    hirugarrena = 1
    for i, box in enumerate(boxes):
      if hirugarrena < 3:
        hirugarrena = hirugarrena+1
        continue
      else:
        #print(box)

        # Obtener el texto dentro de la etiqueta <a> si está presente
        headline_div = box.find('div', class_='content-box-headline')
        if headline_div and headline_div.find('a'):
            box_title = headline_div.find('a').get_text(strip=True)
        else:
            box_title = f'sin_titulo_{i + 1}'

        file_prefix = f'{izena}_{box_title}_{urls}'

        # Encontrar la tabla
        table = box.find('table')

        if table is None:
            continue

        # Crear un archivo CSV para escribir los datos
        csv_filename = os.path.join(output_directory, f'{file_prefix}.csv')
        try:
          with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
              csvwriter = csv.writer(csvfile)

              # Escribir los títulos de las columnas
              csvwriter.writerow(column_titles)

              # Obtener las filas de la tabla
              rows = table.find_all('tr')

              # Iterar sobre las filas y escribir los datos en el archivo CSV
              for row in rows:
                  # Obtener las celdas de la fila
                  cells = row.find_all(['th', 'td'])

                  # Obtener el texto de cada celda y escribirlo en el archivo CSV
                  row_data = [cell.get_text(strip=True) if cell.get_text(strip=True) else '' for cell in cells]
                  csvwriter.writerow(row_data)

          print(f"Los datos de la tabla {i + 1} ({box_title}) se han guardado en '{csv_filename}'.")

          try:
              df = pd.read_csv(csv_filename, header=0)
              #df.to_excel(excel_filename, index=False)
              #print(f"Los datos de la tabla {i + 1} ({box_title}) se han guardado en '{excel_filename}'.")
          except pd.errors.ParserError as e:
              print(f"Error al procesar el archivo CSV para la tabla {i + 1} ({box_title}): {e}")
        except FileNotFoundError:
            print(f"Error: No se pudo encontrar el directorio o el archivo '{csv_filename}'.")
        except Exception as e:
            print(f"Error inesperado: {e}")

In [ ]:
# Inicializar una lista para almacenar la información de todos los jugadores
lista_jugadores = []

# Iterar a través de las URLs y obtener la información de cada jugador
zenbat = 0
for p in PlayerList:
    zenbat = zenbat+1
    print(zenbat)
    print(base+p)
    GetGamesFromPlayer(base+p,PlayerList[p])

In [ ]:
!zip -r datos_tablas.zip datos_tablas/